<a href="https://colab.research.google.com/github/Vasanthi1723/FMML_labs_17/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [12]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [13]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [14]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [15]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [16]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Yes, averaging the validation accuracy across multiple splits, typically done using techniques like cross-validation, generally provides more consistent and reliable results. Here's why:

1. **Reduces Variance**: When you use a single train-test split, the validation accuracy you obtain can be highly dependent on how the data is split. If the data has any inherent bias or imbalance, a single split might not represent the overall performance of the model well. Averaging across multiple splits reduces the variance caused by any single split's peculiarities.

2. **More Representative Performance**: By training and validating on different subsets of the data, you get a more comprehensive view of your model's performance. This is because the model is evaluated on different segments of data, which provides a better approximation of its generalization ability to unseen data.

3. **Mitigates Overfitting**: Using multiple splits can help mitigate the risk of overfitting to a specific validation set. This makes the evaluation more robust, as it ensures that the model performs well across different subsets of the data, not just on a potentially favorable or unfavorable split.

4. **Better Use of Data**: Cross-validation uses all the data for both training and validation. This means that every data point is used for validation exactly once, which helps make the best use of the available da
ta and provides a more thorough evaluation.

In summary, averaging the validation accuracy across multiple splits (like in k-fold cross-validation) generally leads to more consistent and reliable estimates of model performance compared to using a single split.

Yes, averaging validation accuracy across multiple splits generally provides a more accurate estimate of the model's test (or generalization) accuracy. Here’s why:

1. **Reduces Bias and Variance**: By using multiple splits (as in k-fold cross-validation), the model is trained and validated on different portions of the data multiple times. This process reduces the bias that might occur if the model is trained and validated on only one particular split. It also reduces variance because it averages out fluctuations in accuracy caused by any particular split that might be atypical or unrepresentative of the overall dataset.

2. **More Representative of Real-World Performance**: When you average validation accuracy across multiple splits, you obtain a performance measure that better reflects how the model is expected to perform on unseen data. This is because each split acts as a different "scenario" for the model to be tested on, providing a broader perspective on its capabilities.

3. **Less Sensitivity to Outliers**: A single split may contain outliers or an unbalanced distribution of classes, which could skew the validation accuracy. Multiple splits ensure that these anomalies are less likely to disproportionately affect the overall accuracy estimate, as they are spread across several different validation sets.

4. **Balances Data Utilization**: Cross-validation ensures that every data point is used for both training and validation purposes, leading to a more efficient use of the data. This comprehensive usage makes the estimated accuracy more reflective of how the model might perform on a new, unseen dataset.

5. **Generalization to New Data**: Averaging results over multiple splits helps simulate different ways the model could be exposed to new data, providing a better approximation of how well the model generalizes beyond the specific training data.

In conclusion, using multiple splits (such as in cross-validation) gives a more accurate and reliable estimate of the test accuracy because it accounts for variability and provides a comprehensive assessment of the model's performance across different subsets of the data.

The number of iterations (or folds, in the context of cross-validation) has a significant impact on the accuracy and reliability of the estimate of the model's performance. Here’s how the number of iterations affects the estimate:

### Effect of Number of Iterations

1. **Higher Iterations Reduce Variance**: As the number of iterations increases, the variance of the performance estimate tends to decrease. With more iterations, each data point gets to be in both the training and validation sets multiple times, leading to a more stable and less variable estimate of model performance.

2. **Diminishing Returns**: While increasing the number of iterations generally leads to a more accurate estimate of the model’s performance, the benefit tends to diminish beyond a certain point. For example, moving from 5-fold cross-validation to 10-fold cross-validation often provides a more reliable estimate, but moving from 10-fold to 20-fold may not provide a significantly better estimate, especially relative to the increased computational cost.

3. **Computational Cost**: Higher iterations require more computational resources and time. Each iteration involves training the model on a slightly different subset of the data, which can become computationally expensive, especially with large datasets or complex models. Thus, there is a trade-off between computational efficiency and the reliability of the estimate.

4. **Risk of Overfitting to Cross-Validation**: If the number of iterations is very high (like in leave-one-out cross-validation, where the number of iterations equals the number of samples), the model might slightly overfit to the cross-validation process itself, especially if the model is complex and the dataset is small.

### Better Estimates with Higher Iterations?

- **Yes, Up to a Point**: Increasing the number of iterations typically provides a better (more accurate and reliable) estimate of the model's test accuracy because it reduces the influence of any single train-test split. However, this improvement is subject to diminishing returns, where after a certain point, the gain in estimate accuracy becomes marginal compared to the computational effort required.

- **Optimal Number of Iterations**: The optimal number of iterations depends on the size of the dataset, the model's complexity, and the computational resources available. For most practical purposes, using 5 to 10 folds (iterations) in cross-validation is a good balance between reliable performance estimates and computational efficiency.

### Conclusion

Higher iterations generally lead to a better estimate of the model's performance by reducing variance and giving a more comprehensive evaluation across different data splits. However, the benefits decrease after a certain point, and the choice of the number of iterations should balance between accuracy, computational cost, and the potential risk of overfitting to the cross-validation process.

Increasing the number of iterations (or folds) in cross-validation can help to some extent when dealing with a very small training or validation dataset, but there are important considerations and limitations to be aware of:

### Benefits of Increasing Iterations for Small Datasets

1. **Maximizes Data Utilization**: With a small dataset, increasing the number of iterations allows the model to train on as much data as possible. For example, in k-fold cross-validation, each fold serves as both a training set and a validation set. This ensures that every data point is used for training in \(k-1\) folds and for validation in one fold, maximizing the use of limited data.

2. **Reduces Bias and Variance in Estimates**: For small datasets, the variance of performance estimates can be high due to the limited amount of data in each split. More iterations can reduce this variance, leading to more reliable estimates. The increased number of iterations means that the model is exposed to more variations in the training and validation data, which helps in providing a more balanced and less biased estimate of performance.

3. **Mitigates Impact of Small Validation Set**: If the dataset is very small, a single train-test split might result in an unrepresentatively small validation set, which can lead to unreliable performance estimates. Using cross-validation with more iterations ensures that the validation set is varied and not consistently too small or unrepresentative.

### Limitations of Increasing Iterations for Small Datasets

1. **Diminishing Returns**: While increasing iterations can help make better use of a small dataset, there are diminishing returns. For instance, in leave-one-out cross-validation (LOOCV), each data point is used as a validation set once, and all other points are used for training. While this maximizes data utilization, it can be very computationally expensive, especially with complex models, and does not necessarily provide a much better estimate than, say, 10-fold cross-validation.

2. **Risk of Overfitting to Cross-Validation**: With very small datasets, there is a risk of overfitting to the cross-validation process itself. As the model is repeatedly trained on nearly the same data, it might become overly optimized for the specific splits used in cross-validation, potentially leading to less reliable generalization to truly unseen data.

3. **Limited Data Still Means Limited Information**: Even with increased iterations, if the dataset is very small, the model might not have enough data to learn meaningful patterns effectively. The estimates might still be unreliable simply because the dataset doesn't represent the underlying distribution well. Cross-validation can only do so much with limited data — it cannot create new data or information that isn't present in the dataset.

### Conclusion

Increasing the number of iterations in cross-validation can help mitigate some of the issues associated with small training or validation datasets by making better use of the available data and providing more stable performance estimates. However, it is not a perfect solution and comes with limitations such as computational cost and the potential for overfitting to the cross-validation process itself. For very small datasets, it is often more beneficial to seek additional data if possible, use simpler models to prevent overfitting, or apply techniques like data augmentation (in the case of images) or transfer learning, which leverages pre-trained models on larger datasets.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.